In [30]:
from functools import reduce
import datetime

def chksum(nmea:str) -> bool:
    # start index ($)
    start = nmea.find("$")
    # stop index (*)
    stop = nmea.find("*")

    if start == -1 or stop == -1:
        return False

    # substring required for checksum calculation
    substr = nmea[start+1:stop]
    # checksum from sensor
    chksum = nmea[stop+1:]

    return hex(reduce(lambda a,b : a^b, map(ord, substr))).lstrip('0x').upper() == chksum

In [39]:
with open("sample.nmea","r") as f: # opening input file
    with open("latlon.csv","w") as output: # creating output file
        for idx, line in enumerate(f): # reading data line by line with line number (idx)
            if not chksum(line.strip()):
                print(f"checksum error: {idx}")
                continue
            if not "GPGGA" in line: # check if 'GPGGA' is a part of data frame
                continue # go to the next line
            data = line.split(",") # separate parameters, create a list
            ts, lat, lat_dir, lon, lon_dir = data[1:6] # extracting timestamp, latitude and longitude
            lat_dd = float(lat[:2]) + float(lat[2:])/60.0
            lon_dd = float(lon[:3]) + float(lon[3:])/60.0
            if lat_dir == "S": # direction of the latitude, N = North, S = South
                lat_dd *= -1
            if lon_dir == "W": # direction of the longitude, E = East, W = West
                lon_dd *= -1
            ts_obj = datetime.datetime.strptime(ts, "%H%M%S.%f")
            print(f"{ts_obj.strftime('%H:%M:%S')}, {lat_dd:.3f}, {lon_dd:.3f}", file=output)

checksum error: 270
checksum error: 10758
checksum error: 23396


In [6]:
import ipyleaflet

mymap = ipyleaflet.Map(center = [52.55, 13.55], zoom=10)

locations = []
with open("latlon.csv") as f:
    for line in f:
        locations.append(list(map(float, line.split(","))))

# creating new polygon
polyline = ipyleaflet.Polyline(locations = locations, color = "black", fill = False)

mymap.add_layer(polyline)
display(mymap)


Map(center=[52.55, 13.55], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [23]:
nmea = "GPGGA,171120.094,5242.506,N,01300.685,E,1,12,1.0,0.0,M,0.0,M,,"
hex(reduce(lambda a,b : a^b, map(ord, nmea))).lstrip('0x').upper()

'6C'

In [24]:
def xor(a, b):
    return a^b

nmea = "GPGGA,171120.094,5242.506,N,01300.685,E,1,12,1.0,0.0,M,0.0,M,,"
nmea_ord = [ord(char) for char in nmea]
hex(reduce(xor, nmea_ord)).lstrip('0x').upper()

'6C'

In [37]:
time_obj = datetime.datetime.strptime("2024-01-31 12:00:00", "%Y-%m-%d %H:%M:%S")
time_obj = time_obj + datetime.timedelta(hours=-1, seconds = -10)
print(time_obj.strftime("%H:%M:%S.%f %p"))

10:59:50.000000 AM
